In [1]:
import math
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from keras.layers import Dense, Dropout, Activation
from keras.layers import LSTM, GRU
from keras.models import Sequential

lags = 12
config = {"batch": 256, "epochs": 600}
units = [12, 64, 64, 1]

In [2]:
attr = 'speed'
df1 = pd.read_csv('train.csv', encoding='utf-8').fillna(0)
df2 = pd.read_csv('test.csv', encoding='utf-8').fillna(0)
scaler = StandardScaler()
speed1 = scaler.fit_transform(df1[[attr]])
speed2 = scaler.fit_transform(df2[[attr]])

train, test = [], []
for i in range(lags, len(speed1)):
	train.append(speed1[i - lags: i + 1])
for i in range(lags, len(speed2)):
	test.append(speed2[i - lags: i + 1])

train = np.array(train)
test = np.array(test)
np.random.shuffle(train)
X_train = train[:, :-1] # exclude last column
y_train = train[:, -1]
X_test = test[:, :-1]
y_test = test[:, -1]

In [3]:
# model
model = Sequential()
model.add(LSTM(units[1], input_shape=(units[0], 1), return_sequences=True))
model.add(LSTM(units[2]))
model.add(Dropout(0.2))
model.add(Dense(units[3]))

model.compile(loss="mse", optimizer="adam", metrics=['mape'])
hist = model.fit(X_train, y_train, batch_size=config["batch"],epochs=config["epochs"], validation_split=0.05)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/600
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - loss: 0.8897 - mape: 118.3320 - val_loss: 0.5055 - val_mape: 134.5251
Epoch 2/600
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.4616 - mape: 238.5160 - val_loss: 0.5016 - val_mape: 193.6164
Epoch 3/600
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.4493 - mape: 281.0645 - val_loss: 0.4309 - val_mape: 150.7834
Epoch 4/600
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.4313 - mape: 195.5722 - val_loss: 0.4191 - val_mape: 131.0828
Epoch 5/600
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.4188 - mape: 176.3514 - val_loss: 0.3752 - val_mape: 141.7442
Epoch 6/600
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.4003 - mape: 218.8809 - val_loss: 0.3895 - val_mape: 148.2631
Epoch 7/600
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.3878 - mape: 204.5604 - val_loss: 0.3981 - val_mape: 148.7489
Epoch 8/600
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - loss: 0.4212 - mape: 226.6610 - val_loss: 0.3913 - val_mape: 149.4722
Epoch 9/600
7/7 ━━━━━━━━

In [4]:
y_pred = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred)
y_test = scaler.inverse_transform(y_test)

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


In [5]:
# evaluation
mape = metrics.mean_absolute_percentage_error(y_test, y_pred)
mae = metrics.mean_absolute_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
print('MAPE:%f' % mape)
print('MAE:%f' % mae)
print('RMSE:%f' % math.sqrt(mse))

MAPE:0.071027
MAE:4.556106
RMSE:5.987883
